In [1]:
import os
import sys
#
import tensorflow as tf
#tf.keras.backend.set_floatx('float16')
#
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.layers import (
    Activation, AveragePooling2D, BatchNormalization, Conv2D, Conv2DTranspose,
    Dense, Dropout, Flatten, Input, LeakyReLU, ReLU, UpSampling2D)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from time import time

import numpy as np
import cv2
import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras import Input
#from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Conv2D
#
from matplotlib import pyplot
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img,save_img
#from keras.utils.vis_utils import plot_model
from copy import deepcopy

from skimage.color import rgb2lab, lab2rgb
from PIL import Image


2023-09-21 11:44:44.606843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 11:44:45.789681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# conversion lab & rgb
def lab_to_rgb(img_lab):
    img_lab = img_lab.numpy()
    L = (img_lab[:,:,:,0] + 1.) * 50. 
    L = L[..., np.newaxis]
    ab = img_lab[:,:,:,1:]  * 110
    img_lab = np.concatenate((L,ab),axis=3)
    #
    img = lab2rgb(img_lab).astype("float32")
    img = (img * 255).astype(np.uint8)
    return img

def rgb_to_lab(img):
    img_lab = rgb2lab(img).astype("float32") # Converting RGB to L*a*b    # float16 to optimize ?
    # Between -1 and 1
    L  = img_lab[:,:,0]  / 50. - 1. 
    ab = img_lab[:,:,1:]  / 110. # Between -1 and 1 
    return L, ab

In [3]:
# Preload dataset L&ab
# &
# create npz file for GAN input
from os import listdir
from numpy import asarray
from numpy import vstack
from numpy import savez_compressed
import os
from random import shuffle, seed

BATCH_SIZE = 4
#SHUFFLE_BUFFER_SIZE = 100

path_mem = './Dataset_wo_sky_256_modif/V_Urban_Launcher_dts/img_Launcher'
path_old = './Dataset_wo_sky_256_modif/V_Urban_Launcher_dts/img_Urban'
save_path = './Dataset_wo_sky_256_modif/V_Urban_Launcher/'
#
dataset_size = len([i for i in os.listdir(path_old)])
limit = (dataset_size // BATCH_SIZE)*BATCH_SIZE
print(dataset_size,limit)
LIST=[]

i=0

IMAGE=os.listdir(path_mem)
IMAGE.sort()
IMAGE=np.array(IMAGE)

OLD=os.listdir(path_old)
OLD.sort()
OLD=np.array(OLD)

527 524


In [4]:
# Shuffle dataset
seed(1155339977)
shuffle(IMAGE)

seed(112233998877)
shuffle(OLD)

In [6]:
def load_images_path(path):
    im_list= list()
    # enumerate filenames in directory, assume all are images
    for filename in os.listdir(path):
        im_list.append(os.path.join(path,filename))
        print(filename)
    return asarray(im_list)

def load_images(path):
    pixels = Image.open(path).convert("RGB")
    pixels = np.array(pixels).astype("float32")
    pixels = (pixels -127.5) / 127.5
    return np.asarray(pixels)

def create_dataset(LIST,BATCH_SIZE):
    List_dataset = tf.data.Dataset.from_tensor_slices(LIST)
    tf.data.Dataset.save(List_dataset, save_path +'Dataset_'+str(i))
        
    for L in List_dataset:
        print('dataset_'+str(i)+' saved', tf.shape(L))
        break

In [7]:
# Generate dataset file every 100 images 
# because of memory constraints, then 
# restart kernel and merge + save the final file

for filename,filename_old in zip(IMAGE,OLD):
    print(filename,filename_old)
    if ((i % 100 == 0) and (i !=0)):
        create_dataset(LIST,BATCH_SIZE)
        LIST=[]

    # LAB image new
    path = os.path.join(path_mem,filename)
    img_X = load_images(path)
    # old
    path_o = os.path.join(path_old,filename_old)
    img_Y = load_images(path_o)
    #
    LIST.append(np.concatenate((img_X, img_Y), axis=2))
    print(np.concatenate((img_X, img_Y), axis=2).shape)
    i+=1

    if (i==limit):
        create_dataset(LIST,BATCH_SIZE)
        LIST=[]


image_00000046_1.png im_049.JPG
(256, 256, 6)
image_00000187-3.png im_262.png
(256, 256, 6)
image_00000196-3.png im_335.png
(256, 256, 6)
image_00000022-3.png im_327.png
(256, 256, 6)
image_00000017_2.png im_005.jpg
(256, 256, 6)
image_00000152-3.png im_314.png
(256, 256, 6)
image_00000004_4.png im_116.jpg
(256, 256, 6)
image_00000074_2.png gearth_13_09_008_org.jpg
(256, 256, 6)
image_00000013_4.png im_428.png
(256, 256, 6)
image_00000043-3.png im_021.png
(256, 256, 6)
image_00000002_4.png im_403.png
(256, 256, 6)
image_00000046_4.png im_264.png
(256, 256, 6)
image_00000090-3.png im_076.JPG
(256, 256, 6)
image_00000050_1.png im_071.JPG
(256, 256, 6)
image_00000095_1.png im_031.jpg
(256, 256, 6)
image_00000151-3.png im_475.png
(256, 256, 6)
image_00000064_2.png im_395.png
(256, 256, 6)
image_00000599.png im_420.png
(256, 256, 6)
image_00000075-3.png im_193.jpg
(256, 256, 6)
image_00000005_4.png im_050.jpg
(256, 256, 6)
image_00000438.png im_249.png
(256, 256, 6)
image_00000182-3.png im_

2023-09-21 11:45:13.931150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-21 11:45:13.978896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-21 11:45:13.979094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

dataset_100 saved tf.Tensor([256 256   6], shape=(3,), dtype=int32)
(256, 256, 6)
image_00000040-3.png im_194.jpg
(256, 256, 6)
image_00000021-3.png im_381.png
(256, 256, 6)
img_cropped4_14.png gearth_13_09_020_org.jpg
(256, 256, 6)
image_00000011_4.png Madrid_0002063_2014_04_253_40.42918516343075_-3.681197447325949_IpSfL4koGck4xXCpGu5Uzg.jpg
(256, 256, 6)
image_00000041_1.png im_285.png
(256, 256, 6)
image_00000019-3.png im_308.png
(256, 256, 6)
image_00000017.png im_124.jpg
(256, 256, 6)
img_cropped1_2.png im_143.jpg
(256, 256, 6)
image_00000056_2.png im_383.png
(256, 256, 6)
img_cropped4_6.png im_196.jpg
(256, 256, 6)
image_00000094.png im_373.png
(256, 256, 6)
image_00000097_2.png im_468.png
(256, 256, 6)
image_00000002.png im_205.jpg
(256, 256, 6)
img_cropped2_5.png im_176.jpg
(256, 256, 6)
img_cropped2_9.png im_213.jpg
(256, 256, 6)
img_cropped4_2.png im_287.png
(256, 256, 6)
image_00000103-3.png im_141.jpg
(256, 256, 6)
image_00000073-3.png im_033.jpg
(256, 256, 6)
image_0000008

(256, 256, 6)
image_00000027-3.png im_393.png
(256, 256, 6)
img_cropped2_13.png im_164.jpg
(256, 256, 6)
image_00000877.png im_167.jpg
(256, 256, 6)
image_00000092.png im_444.png
(256, 256, 6)
image_00000041_4.png im_099.jpg
(256, 256, 6)
image_00000014.png im_073.JPG
(256, 256, 6)
image_00000064-3.png im_027.jpg
dataset_300 saved tf.Tensor([256 256   6], shape=(3,), dtype=int32)
(256, 256, 6)
image_00000031.png im_353.png
(256, 256, 6)
image_00000070.png im_042.JPG
(256, 256, 6)
image_00000124-3.png im_140.jpg
(256, 256, 6)
image_00000195-3.png im_409.png
(256, 256, 6)
img_cropped1_5.png im_041.jpg
(256, 256, 6)
image_00000055-3.png im_336.png
(256, 256, 6)
image_00000000.png im_464.png
(256, 256, 6)
image_00000082.png im_295.png
(256, 256, 6)
image_00000052.png im_079.jpg
(256, 256, 6)
image_00000014_1.png im_212.jpg
(256, 256, 6)
image_00000023_2.png im_382.png
(256, 256, 6)
image_00000030_2.png im_023.jpg
(256, 256, 6)
image_00000068-3.png gearth_13_09_015_org.jpg
(256, 256, 6)
ima

(256, 256, 6)
image_00000025_4.png im_358.png
(256, 256, 6)
image_00000044_1.png im_047.JPG
(256, 256, 6)
image_00000012_4.png im_097.jpg
(256, 256, 6)
image_00000027.png im_018.png
(256, 256, 6)
image_00000492.png im_120.jpg
(256, 256, 6)
image_00000012.png img_gearth_18_09_001.jpg
(256, 256, 6)
image_00000026_1.png im_064.JPG
(256, 256, 6)
image_00000373.png img_maps_18_09_004.png_256.png
(256, 256, 6)
image_00000130-3.png im_242.png
(256, 256, 6)
image_00000090_1.png im_417.png
(256, 256, 6)
image_00000004.png im_001.jpg
(256, 256, 6)
image_00000048.png im_153.jpg
(256, 256, 6)
image_00000087.png im_236.png
(256, 256, 6)
image_00000041-3.png im_126.jpg
(256, 256, 6)
image_00000011_2.png im_240.png
(256, 256, 6)
image_00000030_1.png im_251.png
dataset_500 saved tf.Tensor([256 256   6], shape=(3,), dtype=int32)
(256, 256, 6)
image_00000086_2.png im_112.jpg
(256, 256, 6)
image_00000098-3.png im_368.png
(256, 256, 6)
image_00000113-3.png im_397.png
(256, 256, 6)
image_00000083.png bb_or

**MERGE**

In [8]:
path_List = './Dataset_wo_sky_256_modif/V_Urban_Launcher/'

List_path=os.listdir(path_List)
List_path.sort()
List_path=np.array(List_path)

d=1

for L in List_path:
    print(L)
    L_data = tf.data.Dataset.load(os.path.join(path_List,L))
    if d: 
        L_dataset = L_data

    else:
        L_dataset = L_dataset.concatenate(L_data)
    d=0
    
print('---------- FIN ----------')
for j,L in enumerate(L_dataset):
    pass
print('Taille finale :',j)

Dataset_100
Dataset_200
Dataset_300
Dataset_400
Dataset_500
Dataset_524
---------- FIN ----------


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7319465687676522997
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 544890511430778091
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing tex

Taille finale : 523


**SAVE**

In [9]:
tf.data.Dataset.save(L_dataset, './Dataset_wo_sky_256_modif/V_Urban_Launcher/Dataset_VUL')

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7319465687676522997
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 2621493309948822991
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing te